<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

BAYES

In [ ]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import numpy as np
import pandas as pd

import scipy.stats as stats
import statsmodels.stats.api as sms

import pylab as plt
import seaborn as sns

import pylab as plt

In [ ]:
# datos
data = 'data/cookie_cats.csv'
df = pd.read_csv(data)
df

In [ ]:
tc1 = df.retention_1[df.version=='gate_30'].value_counts()
tc2 = df.retention_1[df.version=='gate_40'].value_counts()
print(tc1, tc2)

In [ ]:
# 1 dia
from scipy.stats import bernoulli, beta               

df.version.value_counts()

#grupos de testeo y control de 1
imps_ctrl, convs_ctrl = 44700, 20034
imps_test, convs_test= 45489, 20119

a_C, b_C = convs_ctrl+1, imps_ctrl-convs_ctrl+1
a_T, b_T = convs_test+1, imps_test-convs_test+1

beta_control=beta(a_C, b_C)
beta_test=beta(a_T, b_T)

In [ ]:
x=np.linspace(0.43, 0.46, 10000)

def plot(betas, nombres):
    
    for bet, nom in zip(betas, nombres):
        
        y=bet.pdf(x)
        y_media=bet.mean()
        y_std=bet.std()
        
        plt.plot(x, y, label='{}, tasa de conv: {:.6f} $\pm$ {:.10f}'.format(nom, y_media, y_std))
        plt.yticks([])
        
    plt.legend()
    plt.grid()
    plt.show();

In [ ]:
plot([beta_control, beta_test], ['Control', 'Testeo'])

In [ ]:
salto=(beta_test.mean() - beta_control.mean())/beta_control.mean()

salto

In [ ]:
from mpmath import betainc   # integral de beta
from bayes import *

In [ ]:
corte = calc_prob(beta_test, beta_control)

print(corte)

prob=betainc(a_T, b_T, corte, 1, regularized=True)

prob=float(prob)

prob

In [ ]:
'''El test aumenta la tasa de conversion en un {:2.2f}%, con una probabilidad del {:2.2f}%'''.format(salto*100, prob*100)

-----------------------------

In [ ]:
# retention_7
tc3 = df.retention_7[df.version=='gate_30'].value_counts()
tc4 = df.retention_7[df.version=='gate_40'].value_counts()
print(tc3, tc4)

In [ ]:
# 1 dia
from scipy.stats import bernoulli, beta               

df.version.value_counts()

#grupos de testeo y control de 7
imps_ctrl, convs_ctrl = 44700, 8502
imps_test, convs_test= 45489, 8279

a_C, b_C = convs_ctrl+1, imps_ctrl-convs_ctrl+1
a_T, b_T = convs_test+1, imps_test-convs_test+1

beta_control=beta(a_C, b_C)
beta_test=beta(a_T, b_T)

In [ ]:
x=np.linspace(0.16, 0.25, 10000)

def plot2(betas, nombres):
    
    for bet, nom in zip(betas, nombres):
        
        y=bet.pdf(x)
        y_media=bet.mean()
        y_std=bet.std()
        
        plt.plot(x, y, label='{}, tasa de conv: {:.6f} $\pm$ {:.10f}'.format(nom, y_media, y_std))
        plt.yticks([])
        
    plt.legend()
    plt.grid()
    plt.show();
    
plot2([beta_control, beta_test], ['Control', 'Testeo'])

In [ ]:
salto=(beta_test.mean() - beta_control.mean())/beta_control.mean()

salto

In [ ]:
corte = calc_prob(beta_test, beta_control)

print(corte)

prob=betainc(a_T, b_T, corte, 1, regularized=True)

prob=float(prob)

prob

In [ ]:
'''El test aumenta la tasa de conversion en un {:2.2f}%, con una probabilidad del {:2.2f}%'''.format(salto*100, prob*100)

--------------

FRECUENTISMO

In [ ]:
#retention_1

control = df[df.version=='gate_30'].sample(n=44700, random_state=42)

tratamiento = df[df.version=='gate_40'].sample(n=44700, random_state=42)

ab_test = pd.concat([control, tratamiento], axis=0)

ab_test.reset_index(drop=True, inplace=True)

ab_test.head()

In [ ]:
ab_test.version.value_counts()

In [ ]:
tasas_conversion=ab_test.groupby('version')['retention_1']  # agrupo por grupo



std=lambda x: np.std(x, ddof=0)    # std
error_std=lambda x: stats.sem(x, ddof=0)   # error standard , (std/sqrt(n))



tasas_conversion=tasas_conversion.agg([np.mean, std, error_std])
tasas_conversion.columns=['conversion_rate', 'std', 'std_error']


tasas_conversion.style.format('{:.3f}')

In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['version'], 
            y=ab_test['retention_1'])

plt.ylim(0, 0.6)
plt.title('Tasa de conversión por grupo')
plt.xlabel('Grupo')
plt.ylabel('Convertido (proporción)');

In [ ]:
from statsmodels.stats.proportion import proportions_ztest, proportion_confint

In [ ]:
# seleccion de grupos

control_res=ab_test[ab_test.version=='gate_30']['retention_1']

trat_res=ab_test[ab_test.version=='gate_40']['retention_1']

In [ ]:
observaciones = [44700, 44700]  #control_res.count()

conversiones=[control_res.sum(), trat_res.sum()]  # conversiones por grupo

z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones)


# intervalos de confianza
alpha=0.05
(control_a, tratamiento_a), (control_b, tratamiento_b) = proportion_confint(conversiones, 
                                                                            nobs=observaciones,
                                                                            alpha=alpha)

In [ ]:
print(f'z-score: {z_score:.2f}')
print(f'p-valor: {p_valor:.3f}')
print(f'intervalo conf {(1-alpha)*100}% para grupo control: [{control_a:.3f}, {control_b:.3f}]')
print(f'intervalo conf {(1-alpha)*100}% para grupo tratamiento: [{tratamiento_a:.3f}, {tratamiento_b:.3f}]')

In [ ]:
#retention_2
tasas_conversion=ab_test.groupby('version')['retention_7']  # agrupo por grupo



std=lambda x: np.std(x, ddof=0)    # std
error_std=lambda x: stats.sem(x, ddof=0)   # error standard , (std/sqrt(n))



tasas_conversion=tasas_conversion.agg([np.mean, std, error_std])
tasas_conversion.columns=['conversion_rate', 'std', 'std_error']


tasas_conversion.style.format('{:.3f}')

In [ ]:
plt.figure(figsize=(8,6))

sns.barplot(x=ab_test['version'], 
            y=ab_test['retention_7'])

plt.ylim(0, 0.25)
plt.title('Tasa de conversión por grupo')
plt.xlabel('Grupo')
plt.ylabel('Convertido (proporción)');

In [ ]:
# seleccion de grupos

control_res=ab_test[ab_test.version=='gate_30']['retention_7']

trat_res=ab_test[ab_test.version=='gate_40']['retention_7']

In [ ]:
observaciones = [44700, 44700]  #control_res.count()

conversiones=[control_res.sum(), trat_res.sum()]  # conversiones por grupo

z_score, p_valor = proportions_ztest(conversiones, nobs=observaciones)


# intervalos de confianza
alpha=0.05
(control_a, tratamiento_a), (control_b, tratamiento_b) = proportion_confint(conversiones, 
                                                                            nobs=observaciones,
                                                                            alpha=alpha)

In [ ]:
print(f'z-score: {z_score:.2f}')
print(f'p-valor: {p_valor:.3f}')
print(f'intervalo conf {(1-alpha)*100}% para grupo control: [{control_a:.3f}, {control_b:.3f}]')
print(f'intervalo conf {(1-alpha)*100}% para grupo tratamiento: [{tratamiento_a:.3f}, {tratamiento_b:.3f}]')

----

Conclusiones:

De acuerdo con los tests de Bayes realizados, el cambio realizado en la monetización del videojuego no tiene resultados favorables, como muestra el seguimiento que se ha realizado a los jugadores según su retencion.
Sin embargo; al realizar el testeo con frecuentismo, se observa que, a psear de que la retención de los jugadores durante el primer dia no es óptima, el testeo de la retención de los jugadores durante la primera semana nos aporta un pvale inferior al umbral y, por tanto, se puede rechazar la H0 (grupo de control) 